## Part 4: Fraud

In [8]:
%run "../scripts/outlier.py" "../scripts/paths.json"

AnalysisException: Path does not exist: file:/Users/ahirve/Desktop/ads_bnpl/data/curated/postcode.csv

In [9]:
#ETL script not working, so loading the data manually for the time being: 

#Loading consumer fraud data: 
fraud_consumer = pd.read_csv('../data/tables/consumer_fraud_probability.csv')
fraud_consumer = spark.createDataFrame(fraud_consumer) 

#Loading merchant fraud data: 
fraud_merchant = pd.read_csv('../data/tables/merchant_fraud_probabilty.csv')
fraud_merchant = spark.createDataFrame(fraud_merchant) 


NameError: name 'spark' is not defined

The consumer fraud data shows the list of transactions with a high probability of being fraudulent, and therefore should be considered invalid.

In [2]:
print(fraud_consumer.count())
fraud_consumer = fraud_consumer.withColumnRenamed("fraud_probability","cons_fraud_prob").withColumnRenamed("order_datetime","order_datetime_1").withColumnRenamed("user_id","user_id_1")
fraud_consumer.limit(5)

NameError: name 'fraud_consumer' is not defined

Probability of fraud for high revenue merchants...?

In [ ]:
print(fraud_merchants.count())
fraud_merchants = fraud_merchants.withColumnRenamed("merchant_abn","merchant_abn_1").withColumnRenamed("fraud_probability","merch_fraud_prob")
fraud_merchants.limit(5)

In [ ]:

merchant_data = final_join.select("merchant_abn", "categories","revenue_levels","dollar_value","user_id","consumer_id")

- join consumer fraud to final join
- create a version of final join with removed transactions from fraud
- aggregate final join from transaction to merchant level
- add fraud merchant as a feature to aggregated final join
- treat fraud as response and try to create model to give other merchants a fraud score

In [ ]:
#Joining consumer fraud, merchant data, and merchant fraud data to one datatset: 

#Joining consumer fraud and merchant data together

merchant_data.createOrReplaceTempView("temp")

fraud_consumer.createOrReplaceTempView("temp2")

join_1 = spark.sql(""" 

SELECT *
FROM temp

LEFT JOIN temp2

ON temp.user_id = temp2.user_id_1 
""")


In [ ]:
#Joining join_1 to merchant fraud 

#Joining all test + predicting Sets Together 
join_1.createOrReplaceTempView("temp")

fraud_merchants.createOrReplaceTempView("temp2")

final_join = spark.sql(""" 

SELECT *
FROM temp


LEFT JOIN temp2

ON temp.merchant_abn = temp2.merchant_abn_1
""")

In [ ]:
#Changing all null probability values to 0.01: 

MIN_P = 0.01
final_join = final_join.fillna(MIN_P, subset=["cons_fraud_prob", "merch_fraud_prob"])

In [ ]:
#join segment data 
#clustering 
#decide threshold based on clusters 
#use threshold to classify if transaction is fradulent or not 

In [ ]:
# Reading in segments 
segments_data = spark.read.csv('..data/curated/tagged_merchants.csv')

In [ ]:
# segments_data.limit(5)

In [ ]:
#Removing false header line 
segments_data.na.drop(subset=["_c0"])

In [ ]:
final_join.createOrReplaceTempView("temp")

segments_data.createOrReplaceTempView("temp2")

final_join = spark.sql(""" 

SELECT *
FROM temp


LEFT JOIN temp2

ON temp.merchant_abn = temp2._c3
""")

In [ ]:
# final_join.limit(5)

In [ ]:
#Creating a table of average fraud probabilities for each category of purchase: 

aggregated_prob = final_join.groupBy("temp2._c6").agg(avg("cons_fraud_prob").alias("avg_consumer_fraud"),avg("merch_fraud_prob").alias("avg_merchant_fraud"))

In [ ]:
aggregated_prob.show()

In [ ]:
aggregated_prob.limit(6)

In [ ]:
final_join.limit(5)

In [ ]:
#adding a column for is_fraud for each transaction: 

is_fraud = when((final_join["_c6"] == "Electronics") & (final_join["cons_fraud_prob"] > 15.085) &(final_join["merch_fraud_prob"]>29.635),1)\
.when((final_join["_c6"] == "Toys and DIY") & (final_join["cons_fraud_prob"] > 15.843) &(final_join["merch_fraud_prob"]>32.404),1)\
.when((final_join["_c6"] == "Furniture") & (final_join["cons_fraud_prob"] > 15.077) & (final_join["merch_fraud_prob"]>30.941),1)\
.when((final_join["_c6"] == "Beauty, Health, Personal and Household") & (final_join["cons_fraud_prob"] > 15.392) & (final_join["merch_fraud_prob"]>29.76),1)\
.when((final_join["_c6"] == "Electronics") & (final_join["cons_fraud_prob"] > 15.085) &(final_join["merch_fraud_prob"]>29.635),1)\
.when((final_join["_c6"] == "Books, Stationary and Music") & (final_join["cons_fraud_prob"] > 15.140) &(final_join["merch_fraud_prob"]>29.016),1).otherwise(0)



In [ ]:
model_with_fraud = final_join.withColumn("is_fraud",is_fraud)

In [ ]:
model_with_fraud.limit(5)

In [ ]:
model_with_fraud.write.parquet("../data/curated/model_data_with_fraud_col.parquet") 

In [ ]:
#Creating a table of average fraud probabilities for each category of purchase: 
final_model_with_fraud = model_with_fraud.groupBy("merchant_abn_1").agg(avg("is_fraud").alias("average fraud rate per merchant"))

In [ ]:
final_model_with_fraud.limit(20)